In [1]:
import pandas as pd
# !python -m pip install "dask[complete]"  
# !pip install fastparquet
# import dask.dataframe as dd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

path = "/content/drive/MyDrive/ds_portfolio/kaggle_m5/data"

In [2]:
calendar = pd.read_csv(path + "/01_raw/calendar.csv")
sales_train_valid = pd.read_csv(path + "/01_raw/sales_train_validation.csv")
sales_train_eval = pd.read_csv(path + "/01_raw/sales_train_evaluation.csv")
sell_prices = pd.read_csv(path + "/01_raw/sell_prices.csv")
sample_submission = pd.read_csv(path + "/01_raw/sample_submission.csv")

In [4]:
calendar_dummy = pd.concat([calendar, pd.get_dummies(calendar[["event_name_1", "event_type_1", "event_name_2", "event_type_2"]])], axis=1).drop(["weekday", "event_name_1", "event_type_1", "event_name_2", "event_type_2"],axis=1)
calendar_dummy["date"] = pd.to_datetime(calendar_dummy["date"])
calendar_dummy.to_pickle(path + "/02_intermediate/calendar_dummy.pickle")

In [ ]:
all_items = list(sales_train_eval["item_id"].unique())

In [ ]:
for item in tqdm(all_items):
    df = sales_train_eval[sales_train_eval["item_id"]==item]
    df_melted = pd.melt(df, id_vars='id', value_vars=["d_" + str(i) for i in range(1, 1942)])
    df_merged = pd.merge(df_melted, df[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]], on="id", how="right").rename({"variable": "d", "value": "num_of_sales"},axis=1)
    df_calendar = pd.merge(df_merged, calendar_dummy, on="d", how="left")
    df_prices = pd.merge(df_calendar, sell_prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")
    df_prices.to_pickle(path + f"/02_intermediate/items/{item}.pickle")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/3049 [00:00<?, ?it/s]